# Writing A Data Science Blog Post
## 

### By: David Herr
### Dated: August 2nd, 2020

A Jupyter notebook aimed to complete the Udacity Data Science Nanodegree Project 1. In the project, students are to use the CRISP-DM process to understand, prepare, model, and evaluate data to answer real-world business questions.

First, we'll bring in the packages used to complete the project.

In [128]:
# data import and manipulation 
import pandas as pd

# plotting and graphing
%matplotlib inline

# linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# custom modules
import fns_DataVisualization as viz
import fns_DataWrangling as DataWrngl
import fns_TextWrangling as TxtWrngl

#temp library
from sklearn.feature_extraction.text import CountVectorizer

#### 1) Pick A Dataset

Although the traditional CRISP-DM path is to start with business questions, the Udacity Data Science program provided datasets to start with. Thus, we'll start with those datasets and read them in to the notebook.

In [73]:
# Read in Boston dataset
df_boston = pd.read_csv('./_data/listings_boston.csv')

# Read in Seattle dataset
df_seattle = pd.read_csv('./_data/listings_seattle.csv')

# Place dataframes in list for bulk analysis
df_list = [df_boston,df_seattle]


### 2) Pose 3-5 Business Questions
Leveraging this data, we are curious to find out if:

    1. Bedroom or Bathroom count generally correlates to higher pricing?
    2. Are there certain neighborhoods which command generally higher pricing?
    3. Does containing a top N keyword in descriptions result in higher pricing?
    4. Are ratings predictive of pricing?

In [ ]:
#### 3) Use CRISP_DM 
To answer these questions, we'll use what is known as the CRISP_DM process. This process is pictated below:

![Image of CRISP_DM](_img\CRISP_DM.png)


##### 3.1) Business Understanding

**Boston Bed/Bath Analysis**
***

In [ ]:
# define columns to be used for price vs. bed/bath analysis
bed_bath_cols = ['price','bedrooms','bathrooms','beds','bed_type']

# call 2x2 price analysis plot for all bed/bath cols (Boston)
viz.price_analysis_plots(df_boston[bed_bath_cols])

**Boston Neighborhood Analysis**
***

In [ ]:
# define columns to be used for price vs. neighbourhood analysis
neighborhood_cols = ['price','neighbourhood_cleansed']

# call 1x1 price analysis plot for neighborhoods cols (Boston)
viz.price_analysis_plots(df_boston[neighborhood_cols], False)

##### 3.2) Data Understanding

While the head() method is useful, a more thorough understanding of all the available columns, datatypes, as well as profile of that data, is necessary before moving on to preparing the data for modeling.

In [ ]:
# And see the overall shape of each data frame, column names, and column data types
DataWrngl.print_column_types(df_list)

##### 3.3) Data Preperation
In this section, we will be using our above observations to guide both the initial cleansing of our data, as well as initial transformations for feature creation.

**Column Cleansing**
***

We'll start with the data cleansing process on the columns. This will largely involve removing:

- [x]  100% empty
- [x]  Columns with 100% of the same value
- [x]  Database identifications
- [x]  Non-overlapping columns

In [74]:
# Start with dropping columns in all data sets that are 100% na
DataWrngl.drop_na_columns(df_list)

# Define search words to drop common identification column suffixes
search_words = ['_id','_url','_scraped']

# Invoke function to drop id, url, and scraping columns
DataWrngl.drop_ident_columns(df_list,search_words)

# Invoke function to drop any columns containing rows with 100% the same data point
DataWrngl.drop_same_columns(df_list)

# Invoke function to drop non-overlapping columns between two datasets    
DataWrngl.drop_nonoverlap_columns(df_list[0],df_list[1])

# Check to make sure all columns are overlapping between dataframes
sum((df_boston.columns == df_seattle.columns)) / ((len(df_boston.columns) + len(df_seattle.columns)) / 2 ) == 1

True

**Row & Data Cleansing**
***
Next, we'll move into basic *row cleansing* for items we know will break a linear regression model. This includes:

- [x]  True/False - transform to a true boolean rather than text "t" or "f".
- [x]  Substantial Text - must be transformed into a text analytics feature.
- [ ]  Date Columns - drop or transform into a duration if business questions warrant.

In [75]:
# Invoke function to change 't' or 'f' to true Boolean values
DataWrngl.cleanse_data_tf_to_boolean(df_list)

#  Define text columns which require processing for text analytics
text_columns = ['name','summary','space','description','neighborhood_overview', 'notes']

# Invoke tokenization as add columns in both dataframes
TxtWrngl.tokenize_text_columns(df_list,text_columns)

In [133]:

# Leverage pre-tokenized text columns to create bag of words dataframe
tokenized_text_columns = [column + '_tokenized' for column in text_columns]

# define finding top vocab
def get_top_vocabulary(dataframe_list,column_list):

    tokenized_df = pd.DataFrame()

    for dataframe in dataframe_list:
        tokenized_df = tokenized_df.append(dataframe[column_list])

    # initialize count vectorizer object
    vectorizer = CountVectorizer(tokenizer=TxtWrngl.tokenize)

    # get counts of each token (word) in text data
    top_vocab_vector = vectorizer.fit_transform(tokenized_df)

    # Create dataframe to hold vocabulary and count of each item
    top_vocab_df = pd.DataFrame((count,word) for word, count in zip(sum(top_vocab_vector.toarray()).tolist(),vectorizer.get_feature_names()))

    # Assign column names to new dataframe
    top_vocab_df.columns = ['Word', 'Count']

    # Convert the top 20 words to a list for feature extraction
    top_vocab = top_vocab_df.nlargest(20,'Count')['Word'].tolist()

    return top_vocab

result = get_top_vocabulary(df_list,tokenized_text_columns)

result

                                         name_tokenized  \
0                               [sunny, bungalow, city]   
1                  [charming, room, pet, friendly, apt]   
2                          [mexican, folk, art, boston]   
3     [spacious, sunny, bedroom, suite, historic, home]   
4                                  [come, home, boston]   
...                                                 ...   
3813              [3br, mountain, view, house, seattle]   
3814            [portage, bay, view, one, bedroom, apt]   
3815               [private, apartment, view, lake, wa]   
3816                   [amazing, view, modern, comfort]   
3817                      [large, lakefront, apartment]   

                                      summary_tokenized  \
0     [cozy, sunny, family, home, master, bedroom, h...   
1     [charming, quiet, room, second, floor, 1910, c...   
2     [come, stay, friendly, middle, aged, guy, safe...   
3     [come, experience, comfort, home, away, home, ...

##### 3.4) Data Modeling

[Outline placeholder for data modeling section]

##### 3.5) Results Evaluation

[Outline placeholder for evaluation of model and possible re-modeling]

##### 3.6) Deployment

[Outline placeholder for what deployment steps would be]

#### 4. Conclusion

[Outline placeholder for final remarks]